In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
x_train = np.load('/content/drive/My Drive/data.npy')
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
y_train = np.load('/content/drive/My Drive/lable.npy')
y_train = keras.utils.to_categorical(y_train, num_classes = 2)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(1309, 224, 224, 3)
(1309, 2)


In [ ]:
x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
densenet = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

29089792/29084464 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2050      
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________
None


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=100,
    validation_split=0.1
)


Epoch 1/100
19/19 [==============================] - 829s 44s/step - loss: 0.6084 - accuracy: 0.7895 - val_loss: 4.1007 - val_accuracy: 0.2214
Epoch 2/100
19/19 [==============================] - 827s 44s/step - loss: 0.3581 - accuracy: 0.8582 - val_loss: 3.4767 - val_accuracy: 0.2214
Epoch 3/100
19/19 [==============================] - 809s 43s/step - loss: 0.2431 - accuracy: 0.9007 - val_loss: 7.1896 - val_accuracy: 0.2214
Epoch 4/100
19/19 [==============================] - 812s 43s/step - loss: 0.1397 - accuracy: 0.9508 - val_loss: 3.0449 - val_accuracy: 0.4122
Epoch 5/100
19/19 [==============================] - 822s 43s/step - loss: 0.1286 - accuracy: 0.9533 - val_loss: 1.9471 - val_accuracy: 0.5878
Epoch 6/100
19/19 [==============================] - 815s 43s/step - loss: 0.0783 - accuracy: 0.9728 - val_loss: 3.2692 - val_accuracy: 0.4504
Epoch 7/100
19/19 [==============================] - 816s 43s/step - loss: 0.1543 - accuracy: 0.9482 - val_loss: 0.9970 - val_accuracy: 0.8092

In [ ]:
model.save_weights("densenet_weights.h5")

In [ ]:
!ls

densenet_weights.h5  drive  sample_data


In [ ]:
from google.colab import files
files.download('densenet_weights.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
x_test = np.load('/content/drive/My Drive/test_images.npy')
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
y_test = np.load('/content/drive/My Drive/test_labels.npy')
x_test = x_test.astype('float32')
x_test /= 255
x_test.shape

(98, 224, 224, 3)

In [ ]:
result = model.predict_classes(x_test)
print(result)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
